In [1]:
import pandas as pd
import numpy as np 
import datetime as dt
import re
import os
import geojson
import seaborn as sns
from shapely.geometry import shape, GeometryCollection
from shapely.geometry import Polygon, MultiPolygon

import matplotlib.pyplot as plt

from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import SGDRegressor, SGDClassifier
from  math import radians





In [2]:
np.random.seed(0)

In [3]:
initial = pd.read_csv('D:/TOPIC_RELATED VOCABULARY IELTS/Finland/aalto/cemsschool/Other Courses/programming course for DS/project//trip_data/queens2.csv' )


### MODIFYING THE FILE .CHANGE UNIT OF MEASUREMENT
(this step maynot be needed since the original file is parquet and it does not have PUWeekday, Unnamed, etc yet)

In [4]:
initial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1298340 entries, 0 to 1298339
Data columns (total 28 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Unnamed: 0             1298340 non-null  int64  
 1   PUlon                  1298340 non-null  float64
 2   PUlat                  1298340 non-null  float64
 3   DOlon                  1298340 non-null  float64
 4   DOlat                  1298340 non-null  float64
 5   PUmonth                1298340 non-null  int64  
 6   PUhour                 1298340 non-null  int64  
 7   DOmonth                1298340 non-null  int64  
 8   DOhour                 1298340 non-null  int64  
 9   PUweekday              1298340 non-null  object 
 10  DOweekday              1298340 non-null  object 
 11  PUweekend              1298340 non-null  int64  
 12  DOweekend              1298340 non-null  int64  
 13  duration               1298340 non-null  int64  
 14  passenger_count   

In [346]:
initial.describe()

,Unnamed: 0,PUlon,PUlat,DOlon,DOlat,PUmonth,PUhour,DOmonth,DOhour,PUweekend,...,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
count,1.298340e+06,1.298340e+06,1.298340e+06,1.298340e+06,1.298340e+06,1.298340e+06,1.298340e+06,1.298340e+06,1.298340e+06,1.298340e+06,...,1.298340e+06,1.298340e+06,1.298340e+06,1.298340e+06,1.298340e+06,1.298340e+06,1.298340e+06,1.298340e+06,1.298340e+06,1.298340e+06
mean,2.204229e+06,-7.383662e+01,4.070654e+01,-7.392938e+01,4.073712e+01,3.654192e+00,1.438064e+01,3.654299e+00,1.443198e+01,1.518354e-01,...,1.468731e+02,1.425634e+00,3.392440e+03,1.122029e+02,8.747424e+01,4.682713e+02,2.453287e+02,2.945434e-01,4.404383e+03,9.823721e-01
std,1.837215e+06,5.157322e-02,5.681379e-02,6.997592e-02,4.850815e-02,3.443279e+00,5.814822e+00,3.443135e+00,6.034058e+00,3.588614e-01,...,7.431156e+01,5.618795e-01,2.056361e+03,4.388112e+04,4.388093e+04,5.505420e+02,3.593442e+02,5.401880e-02,8.779689e+04,1.224220e+00
min,0.000000e+00,-7.395000e+01,4.056000e+01,-7.423000e+01,4.053000e+01,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,...,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-3.000000e-01,0.000000e+00,-2.500000e+00
25%,6.012758e+05,-7.387000e+01,4.065000e+01,-7.398000e+01,4.071000e+01,1.000000e+00,1.100000e+01,1.000000e+00,1.100000e+01,0.000000e+00,...,8.800000e+01,1.000000e+00,2.000000e+03,0.000000e+00,5.000000e+01,0.000000e+00,0.000000e+00,3.000000e-01,2.330000e+03,0.000000e+00
50%,1.638302e+06,-7.379000e+01,4.069000e+01,-7.395000e+01,4.075000e+01,2.000000e+00,1.500000e+01,2.000000e+00,1.500000e+01,0.000000e+00,...,1.450000e+02,1.000000e+00,3.250000e+03,0.000000e+00,5.000000e+01,3.000000e+02,0.000000e+00,3.000000e-01,4.280000e+03,0.000000e+00
75%,3.646262e+06,-7.379000e+01,4.077000e+01,-7.389000e+01,4.076000e+01,5.000000e+00,1.900000e+01,5.000000e+00,1.900000e+01,0.000000e+00,...,2.190000e+02,2.000000e+00,5.200000e+03,1.000000e+02,5.000000e+01,8.690000e+02,6.120000e+02,3.000000e-01,6.142000e+03,2.500000e+00
max,6.339566e+06,-7.371000e+01,4.079000e+01,-7.371000e+01,4.090000e+01,1.200000e+01,2.300000e+01,1.200000e+01,2.300000e+01,1.000000e+00,...,2.650000e+02,5.000000e+00,1.718500e+05,5.000008e+07,5.000005e+07,5.500000e+04,9.117500e+04,3.000000e-01,1.000004e+08,2.750000e+00


In [348]:
initial.columns


Index(['Unnamed: 0', 'PUlon', 'PUlat', 'DOlon', 'DOlat', 'PUmonth', 'PUhour',
       'DOmonth', 'DOhour', 'PUweekday', 'DOweekday', 'PUweekend', 'DOweekend',
       'duration', 'passenger_count', 'trip_distance', 'RatecodeID',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge'],
      dtype='object')

In [5]:
#USE ONCE AT A TIME AND THEN LOCK IT BY COMMENTING SOMETHING like this(#)
initial[['fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'total_amount']]=initial[['fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'total_amount']]/100

In [350]:
initial

,Unnamed: 0,PUlon,PUlat,DOlon,DOlat,PUmonth,PUhour,DOmonth,DOhour,PUweekday,...,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,859969,-73.93,40.75,-73.98,40.76,1,4,1,4,Mon,...,161,1,10.0,3.0,0.5,2.00,0.00,0.3,15.80,2.5
1,292483,-73.79,40.65,-73.91,40.74,1,19,1,19,Thu,...,260,1,40.0,1.0,0.5,8.36,0.00,0.3,50.16,0.0
2,256705,-73.79,40.65,-73.79,40.65,1,15,1,15,Thu,...,132,2,3.0,0.0,0.5,0.00,0.00,0.3,3.80,0.0
3,428062,-73.79,40.65,-73.93,40.75,1,16,1,17,Fri,...,146,1,44.5,1.0,0.5,9.26,0.00,0.3,55.56,0.0
4,4409936,-73.79,40.65,-73.97,40.80,1,9,1,10,Thu,...,24,1,52.0,0.0,0.5,14.73,6.12,0.3,73.65,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1298335,237507,-73.79,40.65,-73.92,40.70,12,18,12,18,Sat,...,36,2,38.5,0.0,0.5,0.00,0.00,0.3,39.29,0.0
1298336,332982,-73.79,40.65,-73.93,40.69,12,1,12,1,Tue,...,225,3,31.5,0.5,0.5,0.00,0.00,0.3,32.79,0.0
1298337,818217,-73.79,40.65,-73.91,40.78,12,9,12,10,Fri,...,223,1,42.0,0.0,0.5,8.56,0.00,0.3,51.36,0.0
1298338,1139504,-73.87,40.76,-73.95,40.67,12,16,12,17,Fri,...,62,1,36.0,1.0,0.5,7.56,0.00,0.3,45.36,0.0


In [6]:
dd= dict(zip(['Mon','Tue','Wed','Thu','Fri','Sat','Sun'],np.arange(0,7)))


In [7]:
initial['PUday']= initial['PUweekday'].map(dd)
initial['DOday']= initial['DOweekday'].map(dd)
initial['duration_sec']=initial['duration']*60

In [353]:
initial

,Unnamed: 0,PUlon,PUlat,DOlon,DOlat,PUmonth,PUhour,DOmonth,DOhour,PUweekday,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,PUday,DOday,duration_sec
0,859969,-73.93,40.75,-73.98,40.76,1,4,1,4,Mon,...,3.0,0.5,2.00,0.00,0.3,15.80,2.5,0,0,480
1,292483,-73.79,40.65,-73.91,40.74,1,19,1,19,Thu,...,1.0,0.5,8.36,0.00,0.3,50.16,0.0,3,3,1620
2,256705,-73.79,40.65,-73.79,40.65,1,15,1,15,Thu,...,0.0,0.5,0.00,0.00,0.3,3.80,0.0,3,3,60
3,428062,-73.79,40.65,-73.93,40.75,1,16,1,17,Fri,...,1.0,0.5,9.26,0.00,0.3,55.56,0.0,4,4,2280
4,4409936,-73.79,40.65,-73.97,40.80,1,9,1,10,Thu,...,0.0,0.5,14.73,6.12,0.3,73.65,0.0,3,3,2580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1298335,237507,-73.79,40.65,-73.92,40.70,12,18,12,18,Sat,...,0.0,0.5,0.00,0.00,0.3,39.29,0.0,5,5,1740
1298336,332982,-73.79,40.65,-73.93,40.69,12,1,12,1,Tue,...,0.5,0.5,0.00,0.00,0.3,32.79,0.0,1,1,1620
1298337,818217,-73.79,40.65,-73.91,40.78,12,9,12,10,Fri,...,0.0,0.5,8.56,0.00,0.3,51.36,0.0,4,4,1920
1298338,1139504,-73.87,40.76,-73.95,40.67,12,16,12,17,Fri,...,1.0,0.5,7.56,0.00,0.3,45.36,0.0,4,4,1560


In [8]:
initialdropped=initial.drop(['PUweekday','DOweekday','duration'],axis=1)

In [355]:
initialdropped

,Unnamed: 0,PUlon,PUlat,DOlon,DOlat,PUmonth,PUhour,DOmonth,DOhour,PUweekend,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,PUday,DOday,duration_sec
0,859969,-73.93,40.75,-73.98,40.76,1,4,1,4,0,...,3.0,0.5,2.00,0.00,0.3,15.80,2.5,0,0,480
1,292483,-73.79,40.65,-73.91,40.74,1,19,1,19,0,...,1.0,0.5,8.36,0.00,0.3,50.16,0.0,3,3,1620
2,256705,-73.79,40.65,-73.79,40.65,1,15,1,15,0,...,0.0,0.5,0.00,0.00,0.3,3.80,0.0,3,3,60
3,428062,-73.79,40.65,-73.93,40.75,1,16,1,17,0,...,1.0,0.5,9.26,0.00,0.3,55.56,0.0,4,4,2280
4,4409936,-73.79,40.65,-73.97,40.80,1,9,1,10,0,...,0.0,0.5,14.73,6.12,0.3,73.65,0.0,3,3,2580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1298335,237507,-73.79,40.65,-73.92,40.70,12,18,12,18,0,...,0.0,0.5,0.00,0.00,0.3,39.29,0.0,5,5,1740
1298336,332982,-73.79,40.65,-73.93,40.69,12,1,12,1,0,...,0.5,0.5,0.00,0.00,0.3,32.79,0.0,1,1,1620
1298337,818217,-73.79,40.65,-73.91,40.78,12,9,12,10,0,...,0.0,0.5,8.56,0.00,0.3,51.36,0.0,4,4,1920
1298338,1139504,-73.87,40.76,-73.95,40.67,12,16,12,17,0,...,1.0,0.5,7.56,0.00,0.3,45.36,0.0,4,4,1560


In [9]:
initialdropped.isnull().any()

Unnamed: 0               False
PUlon                    False
PUlat                    False
DOlon                    False
DOlat                    False
PUmonth                  False
PUhour                   False
DOmonth                  False
DOhour                   False
PUweekend                False
DOweekend                False
passenger_count          False
trip_distance            False
RatecodeID               False
PULocationID             False
DOLocationID             False
payment_type             False
fare_amount              False
extra                    False
mta_tax                  False
tip_amount               False
tolls_amount             False
improvement_surcharge    False
total_amount             False
congestion_surcharge     False
PUday                    False
DOday                    False
duration_sec             False
dtype: bool

### HANDLING UNREALISTIC DATA AND CATEGORICAL DATA

In [10]:
idxp = initialdropped['payment_type']<=6
idxr=  initialdropped['RatecodeID']<=6

In [11]:
prfiltered= initialdropped[idxp & idxr]

In [12]:
pay= pd.get_dummies(prfiltered['payment_type'],drop_first=True)

In [13]:
payname= pay.rename(columns=dict(zip(np.arange(2,6),['pay2','pay3','pay4','pay5'])))

In [363]:
payname

,pay2,pay3,pay4,pay5
0,0,0,0,0
1,0,0,0,0
2,1,0,0,0
3,0,0,0,0
4,0,0,0,0
...,...,...,...,...
1298335,1,0,0,0
1298336,0,1,0,0
1298337,0,0,0,0
1298338,0,0,0,0


In [14]:
rate= pd.get_dummies(prfiltered['RatecodeID'],drop_first=True)

In [15]:
ratename = rate.rename(columns=dict(zip(np.arange(2,7),['r2','r3','r4','r5','r6'])))

In [366]:

ratename

,r2,r3,r4,r5,r6
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,1,0,0,0,0
...,...,...,...,...,...
1298335,0,0,0,0,0
1298336,0,0,0,0,0
1298337,0,0,0,0,0
1298338,0,0,0,0,0


In [16]:
raw= pd.concat((prfiltered,ratename,payname),axis=1).drop('RatecodeID',axis=1) # keep payment_type column for task 3c

In [17]:
# LOCATION CONDITIONS
idxloc=raw['PULocationID']<=263
idx1loc=raw['DOLocationID']<=263

In [18]:
lofiltered= raw[idxloc & idx1loc]

In [371]:
lofiltered.describe()

,Unnamed: 0,PUlon,PUlat,DOlon,DOlat,PUmonth,PUhour,DOmonth,DOhour,PUweekend,...,duration_sec,r2,r3,r4,r5,r6,pay2,pay3,pay4,pay5
count,1.265917e+06,1.265917e+06,1.265917e+06,1.265917e+06,1.265917e+06,1.265917e+06,1.265917e+06,1.265917e+06,1.265917e+06,1.265917e+06,...,1.265917e+06,1.265917e+06,1.265917e+06,1.265917e+06,1.265917e+06,1.265917e+06,1.265917e+06,1.265917e+06,1.265917e+06,1.265917e+06
mean,2.205857e+06,-7.383708e+01,4.070702e+01,-7.392940e+01,4.073712e+01,3.645182e+00,1.438222e+01,3.645295e+00,1.443577e+01,1.518251e-01,...,1.792838e+03,2.212183e-01,2.927522e-03,9.495093e-04,1.737555e-02,2.606806e-05,3.731943e-01,1.641340e-02,6.367716e-03,7.899412e-07
std,1.837185e+06,5.161682e-02,5.681687e-02,6.997998e-02,4.848738e-02,3.437517e+00,5.810581e+00,3.437385e+00,6.028090e+00,3.588514e-01,...,4.983901e+03,4.150673e-01,5.402735e-02,3.079949e-02,1.306662e-01,5.105624e-03,4.836533e-01,1.270591e-01,7.954353e-02,8.887864e-04
min,0.000000e+00,-7.395000e+01,4.056000e+01,-7.423000e+01,4.053000e+01,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,6.024580e+05,-7.387000e+01,4.065000e+01,-7.398000e+01,4.071000e+01,1.000000e+00,1.100000e+01,1.000000e+00,1.100000e+01,0.000000e+00,...,9.000000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.641328e+06,-7.381000e+01,4.071000e+01,-7.395000e+01,4.075000e+01,2.000000e+00,1.500000e+01,2.000000e+00,1.500000e+01,0.000000e+00,...,1.500000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,3.647832e+06,-7.379000e+01,4.077000e+01,-7.389000e+01,4.076000e+01,5.000000e+00,1.900000e+01,5.000000e+00,1.900000e+01,0.000000e+00,...,2.160000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,6.339566e+06,-7.371000e+01,4.079000e+01,-7.371000e+01,4.090000e+01,1.200000e+01,2.300000e+01,1.200000e+01,2.300000e+01,1.000000e+00,...,2.847720e+06,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [19]:
#DEALING WITH FEATURES THAT ARE ONLY MEANINGFUL WHEN IT IS >=0
    #APPROACH 2 (Absolute values) 
    #### REPRODUCIBLE OR NOT COlumns when stating columns from 5 to the end???
lonilfiltered=np.abs(lofiltered.iloc[:,5::])
lonilfiltered0= pd.concat((lofiltered.iloc[:,0:5],lonilfiltered),axis=1)



In [373]:
lonilfiltered.describe()

,PUmonth,PUhour,DOmonth,DOhour,PUweekend,DOweekend,passenger_count,trip_distance,PULocationID,DOLocationID,...,duration_sec,r2,r3,r4,r5,r6,pay2,pay3,pay4,pay5
count,1.265917e+06,1.265917e+06,1.265917e+06,1.265917e+06,1.265917e+06,1.265917e+06,1.265917e+06,1.265917e+06,1.265917e+06,1.265917e+06,...,1.265917e+06,1.265917e+06,1.265917e+06,1.265917e+06,1.265917e+06,1.265917e+06,1.265917e+06,1.265917e+06,1.265917e+06,1.265917e+06
mean,3.645182e+00,1.438222e+01,3.645295e+00,1.443577e+01,1.518251e-01,1.508843e-01,1.465188e+00,1.047788e+01,1.354566e+02,1.438687e+02,...,1.792838e+03,2.212183e-01,2.927522e-03,9.495093e-04,1.737555e-02,2.606806e-05,3.731943e-01,1.641340e-02,6.367716e-03,7.899412e-07
std,3.437517e+00,5.810581e+00,3.437385e+00,6.028090e+00,3.588514e-01,3.579362e-01,1.095114e+00,1.581328e+01,3.444858e+01,7.280076e+01,...,4.983901e+03,4.150673e-01,5.402735e-02,3.079949e-02,1.306662e-01,5.105624e-03,4.836533e-01,1.270591e-01,7.954353e-02,8.887864e-04
min,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.000000e+00,1.100000e+01,1.000000e+00,1.100000e+01,0.000000e+00,0.000000e+00,1.000000e+00,5.310000e+00,1.320000e+02,8.700000e+01,...,9.000000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.000000e+00,1.500000e+01,2.000000e+00,1.500000e+01,0.000000e+00,0.000000e+00,1.000000e+00,9.800000e+00,1.320000e+02,1.450000e+02,...,1.500000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,5.000000e+00,1.900000e+01,5.000000e+00,1.900000e+01,0.000000e+00,0.000000e+00,2.000000e+00,1.653000e+01,1.380000e+02,2.150000e+02,...,2.160000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,1.200000e+01,2.300000e+01,1.200000e+01,2.300000e+01,1.000000e+00,1.000000e+00,9.000000e+00,1.616126e+04,2.600000e+02,2.630000e+02,...,2.847720e+06,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [20]:
#DEALING WITH FEATURES THAT ARE ONLY MEANINGFUL WHEN IT IS >0
    #APPROACH 1 
#DURATIOIN CONDITION
c01=  lonilfiltered0['duration_sec']>0
# PASSENGER COUNT CONDITION 
c02= lonilfiltered0['passenger_count']>0
#TRIP DISTANCE CONDITION
c03= lonilfiltered0['trip_distance']>0

lonilfiltered= lonilfiltered0[c01 & c02 &  c03]

In [21]:
lonilfiltered


,Unnamed: 0,PUlon,PUlat,DOlon,DOlat,PUmonth,PUhour,DOmonth,DOhour,PUweekend,...,duration_sec,r2,r3,r4,r5,r6,pay2,pay3,pay4,pay5
0,859969,-73.93,40.75,-73.98,40.76,1.0,4.0,1.0,4.0,0.0,...,480.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,292483,-73.79,40.65,-73.91,40.74,1.0,19.0,1.0,19.0,0.0,...,1620.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,256705,-73.79,40.65,-73.79,40.65,1.0,15.0,1.0,15.0,0.0,...,60.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,428062,-73.79,40.65,-73.93,40.75,1.0,16.0,1.0,17.0,0.0,...,2280.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4409936,-73.79,40.65,-73.97,40.80,1.0,9.0,1.0,10.0,0.0,...,2580.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1298335,237507,-73.79,40.65,-73.92,40.70,12.0,18.0,12.0,18.0,0.0,...,1740.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1298336,332982,-73.79,40.65,-73.93,40.69,12.0,1.0,12.0,1.0,0.0,...,1620.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1298337,818217,-73.79,40.65,-73.91,40.78,12.0,9.0,12.0,10.0,0.0,...,1920.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1298338,1139504,-73.87,40.76,-73.95,40.67,12.0,16.0,12.0,17.0,0.0,...,1560.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
# Speed conditions
sc1= (lonilfiltered['trip_distance']/lonilfiltered['duration_sec'])<=0.009 #17mil JFK to timesquare 45 min # < 34MPH 
sc2= (lonilfiltered['trip_distance']/lonilfiltered['duration_sec'])>0.00097 # >3.5MPH
lonilspfiltered= lonilfiltered[sc1 & sc2]

In [377]:
lonilspfiltered.describe()

,Unnamed: 0,PUlon,PUlat,DOlon,DOlat,PUmonth,PUhour,DOmonth,DOhour,PUweekend,...,duration_sec,r2,r3,r4,r5,r6,pay2,pay3,pay4,pay5
count,8.934710e+05,893471.000000,893471.000000,893471.000000,893471.000000,893471.000000,893471.000000,893471.000000,893471.000000,893471.000000,...,893471.000000,893471.000000,893471.000000,893471.000000,893471.000000,893471.000000,893471.000000,893471.000000,893471.000000,893471.0
mean,2.294525e+06,-73.843575,40.716162,-73.934650,40.738890,3.458548,14.389364,3.458679,14.609391,0.135369,...,1712.737559,0.196323,0.001532,0.000694,0.007438,0.000009,0.372406,0.008789,0.004606,0.0
std,1.856178e+06,0.049933,0.056583,0.065787,0.044684,3.340700,5.320332,3.340604,5.480605,0.342117,...,1009.097627,0.397216,0.039114,0.026333,0.085925,0.002992,0.483446,0.093338,0.067708,0.0
min,0.000000e+00,-73.950000,40.560000,-74.230000,40.530000,1.000000,0.000000,1.000000,0.000000,0.000000,...,60.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,6.652300e+05,-73.870000,40.650000,-73.980000,40.710000,1.000000,11.000000,1.000000,11.000000,0.000000,...,960.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
50%,1.777752e+06,-73.870000,40.750000,-73.960000,40.750000,2.000000,15.000000,2.000000,16.000000,0.000000,...,1560.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
75%,3.806750e+06,-73.790000,40.770000,-73.910000,40.760000,3.000000,18.000000,3.000000,19.000000,0.000000,...,2340.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0
max,6.339529e+06,-73.710000,40.790000,-73.710000,40.900000,12.000000,23.000000,12.000000,23.000000,1.000000,...,92400.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0


In [23]:
# FARE AMOUNT CONDITIONS

fc1= lonilspfiltered['fare_amount']<=lonilspfiltered['trip_distance']*3+ ((lonilspfiltered['trip_distance']*0.5)/0.2)
fc2= lonilspfiltered['fare_amount']>lonilspfiltered['trip_distance']*2.5
lonilspffiltered= lonilspfiltered[fc1 & fc2]

In [24]:
lonilspffiltered

,Unnamed: 0,PUlon,PUlat,DOlon,DOlat,PUmonth,PUhour,DOmonth,DOhour,PUweekend,...,duration_sec,r2,r3,r4,r5,r6,pay2,pay3,pay4,pay5
0,859969,-73.93,40.75,-73.98,40.76,1.0,4.0,1.0,4.0,0.0,...,480.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,292483,-73.79,40.65,-73.91,40.74,1.0,19.0,1.0,19.0,0.0,...,1620.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,428062,-73.79,40.65,-73.93,40.75,1.0,16.0,1.0,17.0,0.0,...,2280.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4409936,-73.79,40.65,-73.97,40.80,1.0,9.0,1.0,10.0,0.0,...,2580.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,3366728,-73.87,40.77,-73.79,40.65,1.0,20.0,1.0,21.0,0.0,...,1320.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1298335,237507,-73.79,40.65,-73.92,40.70,12.0,18.0,12.0,18.0,0.0,...,1740.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1298336,332982,-73.79,40.65,-73.93,40.69,12.0,1.0,12.0,1.0,0.0,...,1620.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1298337,818217,-73.79,40.65,-73.91,40.78,12.0,9.0,12.0,10.0,0.0,...,1920.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1298338,1139504,-73.87,40.76,-73.95,40.67,12.0,16.0,12.0,17.0,0.0,...,1560.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
raw= lonilspffiltered

In [382]:
raw.columns

Index(['Unnamed: 0', 'PUlon', 'PUlat', 'DOlon', 'DOlat', 'PUmonth', 'PUhour',
       'DOmonth', 'DOhour', 'PUweekend', 'DOweekend', 'passenger_count',
       'trip_distance', 'PULocationID', 'DOLocationID', 'payment_type',
       'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount', 'congestion_surcharge',
       'PUday', 'DOday', 'duration_sec', 'r2', 'r3', 'r4', 'r5', 'r6', 'pay2',
       'pay3', 'pay4', 'pay5'],
      dtype='object')

### AUGMENTED DATASET AND DOING THE TRANSOFMATION FOR PERIODICITY FEATURES

In [26]:
#SIN COS TRANSFORM FOR MONTH DAY HOUR
raw['PUhoursin']= np.sin(raw['PUhour']*(2.*np.pi/24)) # coordinate values on Oy
raw['PUhourcos']= np.cos(raw['PUhour']*(2.*np.pi/24)) #coordinate values on Ox
raw['DOhoursin']= np.sin(raw['DOhour']*(2.*np.pi/24)) # coordinate values on Oy
raw['DOhourcos']= np.cos(raw['DOhour']*(2.*np.pi/24)) #coordinate values on Ox

raw['PUdaysin']= np.sin(raw['PUday']*(2.*np.pi/7))
raw['PUdaycos']= np.cos(raw['PUday']*(2.*np.pi/7))
raw['DOdaysin']= np.sin(raw['DOday']*(2.*np.pi/7))
raw['DOdaycos']= np.cos(raw['DOday']*(2.*np.pi/7))


raw['PUmonthsin']= np.sin(raw['PUmonth']*(2.*np.pi/12))
raw['PUmonthcos']= np.cos(raw['PUmonth']*(2.*np.pi/12))
raw['DOmonthsin']= np.sin(raw['DOmonth']*(2.*np.pi/12))
raw['DOmonthcos']= np.cos(raw['DOmonth']*(2.*np.pi/12))


<ipython-input-26-5ef35a10511a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw['PUhoursin']= np.sin(raw['PUhour']*(2.*np.pi/24)) # coordinate values on Oy
<ipython-input-26-5ef35a10511a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw['PUhourcos']= np.cos(raw['PUhour']*(2.*np.pi/24)) #coordinate values on Ox
<ipython-input-26-5ef35a10511a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

In [27]:
raw.drop(['PUhour','DOhour','PUday','DOday','PUmonth','DOmonth'],axis=1,inplace=True)

C:\Users\ADMIN\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [385]:
raw.columns

Index(['Unnamed: 0', 'PUlon', 'PUlat', 'DOlon', 'DOlat', 'PUweekend',
       'DOweekend', 'passenger_count', 'trip_distance', 'PULocationID',
       'DOLocationID', 'payment_type', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount',
       'congestion_surcharge', 'duration_sec', 'r2', 'r3', 'r4', 'r5', 'r6',
       'pay2', 'pay3', 'pay4', 'pay5', 'PUhoursin', 'PUhourcos', 'DOhoursin',
       'DOhourcos', 'PUdaysin', 'PUdaycos', 'DOdaysin', 'DOdaycos',
       'PUmonthsin', 'PUmonthcos', 'DOmonthsin', 'DOmonthcos'],
      dtype='object')

In [28]:
len(raw)

809612

### HANDLING LONGITUDE LATITUDE FEATURES BASED ON LOCATIONID

In [29]:
###LON LAT OF SOME SPECIFIC PLACES
JFKairport= (-73.780968, 40.641766)

#Queens library = 40.757978766519294, -73.82900393688718
Queenslib = -73.83, 40.76 

#LGDairport= 40.776902684722835, -73.87395517203288
LGDairport= -73.87, 40.78 

#marinomar= 40.77123716721885, -73.80129679098286
Marinomar= -73.80, 40.77 

L= [JFKairport,LGDairport, Queenslib, Marinomar]


In [30]:
#Converting decimal lon lat to radians for each specific places mentioned above
def detora(List):
    res=[]
    for i in List:
        listc= list(map(radians,i))
        res.append(listc)
    return res


In [31]:
res= detora(L)

In [32]:
res

[[-1.2877208169085756, 0.7093326305250857],
 [-1.2892747184482114, 0.7117452689632876],
 [-1.2885765867474135, 0.7113962031128886],
 [-1.2880529879718152, 0.7115707360380882]]

In [33]:
# convert decimal lon lat to radians for PUlon  PUlat , DOlon, DOlat
raw['PUlonra']= np.radians(raw['PUlon'])
raw['PUlatra']= np.radians(raw['PUlat'])
raw['DOlonra']= np.radians(raw['DOlon'])
raw['DOlatra']= np.radians(raw['DOlat'])


<ipython-input-33-29ab1a99c552>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw['PUlonra']= np.radians(raw['PUlon'])
<ipython-input-33-29ab1a99c552>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw['PUlatra']= np.radians(raw['PUlat'])
<ipython-input-33-29ab1a99c552>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

In [34]:
#Compute the Harvesine distance between the datapoints and each specific place listed above
def haversineindf(df):
    for i in np.arange(0,len(res)):
        dlonp= raw['PUlonra']-res[i][0]
        dlatp= raw['PUlatra']-res[i][1]
        ap= np.sin(dlatp/2)**2 + np.cos(raw['PUlatra']) * np.cos(res[i][1]) * np.sin(dlonp/2)**2
        cp= 2*np.arcsin(np.sqrt(ap))
        sp=cp*3956
        sp=sp.rename(i)
        df= pd.concat((df,sp),axis=1)
    return df

In [35]:
#Compute the Harvesine distance between the PU DO 
def haversineindfpd(df):
    dlonpd= raw['PUlonra']-raw['DOlonra']
    dlatpd= raw['PUlatra']-raw['DOlatra']
    apd= np.sin(dlatpd/2)**2 + np.cos(raw['PUlatra']) * np.cos(raw['DOlatra']) * np.sin(dlonpd/2)**2
    cpd= 2*np.arcsin(np.sqrt(apd))
    spd=cpd*3956
    df= pd.concat((df,spd),axis=1)
    return df

In [36]:
s1= pd.DataFrame(np.zeros(len(raw.index)),index=raw.index)
predf= haversineindf(s1)
predfpd= haversineindfpd(s1)

In [37]:
predf

,0,0,1,2,3
0,0.0,10.803330,3.759717,5.275608,6.937609
1,0.0,0.739663,9.904285,7.878266,8.301941
3,0.0,0.739663,9.904285,7.878266,8.301941
4,0.0,0.739663,9.904285,7.878266,8.301941
6,0.0,10.005403,0.690452,2.202784,3.660336
...,...,...,...,...,...
1298335,0.0,0.739663,9.904285,7.878266,8.301941
1298336,0.0,0.739663,9.904285,7.878266,8.301941
1298337,0.0,0.739663,9.904285,7.878266,8.301941
1298338,0.0,9.400086,1.380905,2.091935,3.725157


In [38]:
df=predf.iloc[:,1::]
dfpd=predfpd.iloc[:,1::]

In [39]:
df.describe()

,0,1,2,3
count,809612.000000,809612.000000,809612.000000,809612.000000
mean,5.995595,5.091263,4.983647,6.002257
std,4.614038,4.319308,2.670024,2.204717
min,0.739663,0.690452,0.000000,0.522905
25%,0.739663,0.690452,2.202784,3.660336
50%,9.949019,3.759717,4.905918,6.313202
75%,10.005403,9.904285,7.878266,8.301941
max,12.001798,15.333708,14.430769,15.602078


In [40]:
dfname=df.rename(columns=dict(zip(np.arange(0,4),['tojfk','tolgd', 'toql','tomar'])))
dfpdname=dfpd.rename(columns={0:'pd'})

In [41]:
Xfull= pd.concat((raw,dfname, dfpdname),axis=1)

In [42]:
Xfull.drop(['PUlon','PUlat','DOlon','DOlat', 'PUlonra','PUlatra','DOlonra','DOlatra','PULocationID','DOLocationID' ],axis=1,inplace=True)

In [43]:
Xfull

,Unnamed: 0,PUweekend,DOweekend,passenger_count,trip_distance,payment_type,fare_amount,extra,mta_tax,tip_amount,...,DOdaycos,PUmonthsin,PUmonthcos,DOmonthsin,DOmonthcos,tojfk,tolgd,toql,tomar,pd
0,859969,0.0,0.0,1.0,2.70,1.0,10.0,3.0,0.5,2.00,...,1.000000,5.000000e-01,0.866025,5.000000e-01,0.866025,10.803330,3.759717,5.275608,6.937609,2.704728
1,292483,0.0,0.0,1.0,14.53,1.0,40.0,1.0,0.5,8.36,...,-0.900969,5.000000e-01,0.866025,5.000000e-01,0.866025,0.739663,9.904285,7.878266,8.301941,8.836141
3,428062,0.0,0.0,2.0,15.40,1.0,44.5,1.0,0.5,9.26,...,-0.900969,5.000000e-01,0.866025,5.000000e-01,0.866025,0.739663,9.904285,7.878266,8.301941,10.068639
4,4409936,0.0,0.0,1.0,19.77,1.0,52.0,0.0,0.5,14.73,...,-0.900969,5.000000e-01,0.866025,5.000000e-01,0.866025,0.739663,9.904285,7.878266,8.301941,13.999071
6,3366728,0.0,0.0,1.0,11.51,2.0,32.0,0.5,0.5,0.00,...,-0.900969,5.000000e-01,0.866025,5.000000e-01,0.866025,10.005403,0.690452,2.202784,3.660336,9.283285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1298335,237507,0.0,0.0,1.0,13.10,2.0,38.5,0.0,0.5,0.00,...,-0.222521,-2.449294e-16,1.000000,-2.449294e-16,1.000000,0.739663,9.904285,7.878266,8.301941,7.632812
1298336,332982,0.0,0.0,1.0,10.80,3.0,31.5,0.5,0.5,0.00,...,0.623490,-2.449294e-16,1.000000,-2.449294e-16,1.000000,0.739663,9.904285,7.878266,8.301941,7.834607
1298337,818217,0.0,0.0,1.0,14.94,1.0,42.0,0.0,0.5,8.56,...,-0.900969,-2.449294e-16,1.000000,-2.449294e-16,1.000000,0.739663,9.904285,7.878266,8.301941,10.954698
1298338,1139504,0.0,0.0,1.0,12.04,1.0,36.0,1.0,0.5,7.56,...,-0.900969,-2.449294e-16,1.000000,-2.449294e-16,1.000000,9.400086,1.380905,2.091935,3.725157,7.492872


In [44]:
Xfull.columns

Index(['Unnamed: 0', 'PUweekend', 'DOweekend', 'passenger_count',
       'trip_distance', 'payment_type', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount',
       'congestion_surcharge', 'duration_sec', 'r2', 'r3', 'r4', 'r5', 'r6',
       'pay2', 'pay3', 'pay4', 'pay5', 'PUhoursin', 'PUhourcos', 'DOhoursin',
       'DOhourcos', 'PUdaysin', 'PUdaycos', 'DOdaysin', 'DOdaycos',
       'PUmonthsin', 'PUmonthcos', 'DOmonthsin', 'DOmonthcos', 'tojfk',
       'tolgd', 'toql', 'tomar', 'pd'],
      dtype='object')

### TRIMMING COLUMNS THAT WOULD NOT BE USED AT ALL FOR TASK 3 WHICH HAS NOT BEEN DEALT WITH FROM THE OUTSET
(NOte:Whenever I create a new column based on the available columns from the original table, I dropped those columns given that the newly created column would act as an useful alternative for the dropped columns already. However, there would be still  the Unnamed columns that were useful above still existing the table. Now, it is time to say goodbye to them)

In [45]:
Xfull.columns

Index(['Unnamed: 0', 'PUweekend', 'DOweekend', 'passenger_count',
       'trip_distance', 'payment_type', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount',
       'congestion_surcharge', 'duration_sec', 'r2', 'r3', 'r4', 'r5', 'r6',
       'pay2', 'pay3', 'pay4', 'pay5', 'PUhoursin', 'PUhourcos', 'DOhoursin',
       'DOhourcos', 'PUdaysin', 'PUdaycos', 'DOdaysin', 'DOdaycos',
       'PUmonthsin', 'PUmonthcos', 'DOmonthsin', 'DOmonthcos', 'tojfk',
       'tolgd', 'toql', 'tomar', 'pd'],
      dtype='object')

In [46]:
Xfull['pay5'].unique() # since there is no variability of this feature in dataset

array([0.])

In [47]:
Xfull.drop(['Unnamed: 0','pay5'],axis=1, inplace=True)

In [48]:
Xuse=Xfull.loc[:,~Xfull.columns.duplicated()]

In [49]:
Xuse.isnull().any()

PUweekend                False
DOweekend                False
passenger_count          False
trip_distance            False
payment_type             False
fare_amount              False
extra                    False
mta_tax                  False
tip_amount               False
tolls_amount             False
improvement_surcharge    False
total_amount             False
congestion_surcharge     False
duration_sec             False
r2                       False
r3                       False
r4                       False
r5                       False
r6                       False
pay2                     False
pay3                     False
pay4                     False
PUhoursin                False
PUhourcos                False
DOhoursin                False
DOhourcos                False
PUdaysin                 False
PUdaycos                 False
DOdaysin                 False
DOdaycos                 False
PUmonthsin               False
PUmonthcos               False
DOmonths

### SCALING DATA

In [50]:
#why scaling: convergence and all features use the same scale to elimination the dominance of some features over others
rs=RobustScaler()
#why robustscaler: outliers in trip_distance

In [51]:
Xuse['trip_distance'].describe() 

count    809612.000000
mean         10.433654
std           5.484775
min           0.400000
25%           6.600000
50%           9.700000
75%          14.610000
max          81.400000
Name: trip_distance, dtype: float64

In [52]:
rs.fit(Xuse)

RobustScaler()

In [54]:
Xscaled= pd.DataFrame(rs.transform(Xuse),index=Xuse.index,columns=Xuse.columns)


In [55]:
Xscaled

,PUweekend,DOweekend,passenger_count,trip_distance,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,...,DOdaycos,PUmonthsin,PUmonthcos,DOmonthsin,DOmonthcos,tojfk,tolgd,toql,tomar,pd
0,0.0,0.0,0.0,-0.873908,0.0,-0.857143,3.0,0.0,-0.239234,0.0,...,8.019377e-01,0.000000,0.42265,0.000000,0.42265,0.092201,0.000000,0.065138,0.134524,-0.700253
1,0.0,0.0,0.0,0.602996,0.0,0.367347,1.0,0.0,0.521531,0.0,...,-4.450419e-01,0.000000,0.42265,0.000000,0.42265,-0.993915,0.666885,0.523717,0.428459,0.493520
3,0.0,0.0,1.0,0.711610,0.0,0.551020,1.0,0.0,0.629187,0.0,...,-4.450419e-01,0.000000,0.42265,0.000000,0.42265,-0.993915,0.666885,0.523717,0.428459,0.733484
4,0.0,0.0,0.0,1.257179,0.0,0.857143,0.0,0.0,1.283493,1.0,...,-4.450419e-01,0.000000,0.42265,0.000000,0.42265,-0.993915,0.666885,0.523717,0.428459,1.498731
6,0.0,0.0,0.0,0.225968,1.0,0.040816,0.5,0.0,-0.478469,0.0,...,-4.450419e-01,0.000000,0.42265,0.000000,0.42265,0.006085,-0.333115,-0.476283,-0.571541,0.580578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1298335,0.0,0.0,0.0,0.424469,1.0,0.306122,0.0,0.0,-0.478469,0.0,...,-1.638616e-16,-1.366025,0.57735,-1.366025,0.57735,-0.993915,0.666885,0.523717,0.428459,0.259234
1298336,0.0,0.0,0.0,0.137328,2.0,0.020408,0.5,0.0,-0.478469,0.0,...,5.549581e-01,-1.366025,0.57735,-1.366025,0.57735,-0.993915,0.666885,0.523717,0.428459,0.298523
1298337,0.0,0.0,0.0,0.654182,0.0,0.448980,0.0,0.0,0.545455,0.0,...,-4.450419e-01,-1.366025,0.57735,-1.366025,0.57735,-0.993915,0.666885,0.523717,0.428459,0.905998
1298338,0.0,0.0,0.0,0.292135,0.0,0.204082,1.0,0.0,0.425837,0.0,...,-4.450419e-01,-1.366025,0.57735,-1.366025,0.57735,-0.059243,-0.258178,-0.495814,-0.557575,0.231988


### TASK 3

#### 3a Predicting trip distance

In [56]:

Xscaled.columns

Index(['PUweekend', 'DOweekend', 'passenger_count', 'trip_distance',
       'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount',
       'tolls_amount', 'improvement_surcharge', 'total_amount',
       'congestion_surcharge', 'duration_sec', 'r2', 'r3', 'r4', 'r5', 'r6',
       'pay2', 'pay3', 'pay4', 'PUhoursin', 'PUhourcos', 'DOhoursin',
       'DOhourcos', 'PUdaysin', 'PUdaycos', 'DOdaysin', 'DOdaycos',
       'PUmonthsin', 'PUmonthcos', 'DOmonthsin', 'DOmonthcos', 'tojfk',
       'tolgd', 'toql', 'tomar', 'pd'],
      dtype='object')

In [57]:
ya=Xscaled['trip_distance']

In [58]:
Xa= Xscaled.drop(['trip_distance','pd','duration_sec','passenger_count','payment_type','pay2','pay3','pay4','r2','r3','r4','r5','r6',
       'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount', 'congestion_surcharge',
       'DOweekend','DOhoursin', 'DOhourcos','DOdaysin','DOdaycos','DOmonthsin','DOmonthcos'],axis=1)


In [59]:
Xa.columns

Index(['PUweekend', 'PUhoursin', 'PUhourcos', 'PUdaysin', 'PUdaycos',
       'PUmonthsin', 'PUmonthcos', 'tojfk', 'tolgd', 'toql', 'tomar'],
      dtype='object')

In [60]:
Xa

,PUweekend,PUhoursin,PUhourcos,PUdaysin,PUdaycos,PUmonthsin,PUmonthcos,tojfk,tolgd,toql,tomar
0,0.0,1.577350e+00,0.674599,0.000000,8.019377e-01,0.000000,0.42265,0.092201,0.000000,0.065138,0.134524
1,0.0,-5.380048e-01,0.460186,0.356896,-4.450419e-01,0.000000,0.42265,-0.993915,0.666885,0.523717,0.428459
3,0.0,-4.226497e-01,-0.214413,-0.356896,-4.450419e-01,0.000000,0.42265,-0.993915,0.666885,0.523717,0.428459
4,0.0,1.393847e+00,-0.398533,0.356896,-4.450419e-01,0.000000,0.42265,-0.993915,0.666885,0.523717,0.428459
6,0.0,-4.226497e-01,0.674599,-0.356896,-4.450419e-01,0.000000,0.42265,0.006085,-0.333115,-0.476283,-0.571541
...,...,...,...,...,...,...,...,...,...,...,...
1298335,0.0,-5.773503e-01,0.230093,-0.801938,-1.638616e-16,-1.366025,0.57735,-0.993915,0.666885,0.523717,0.428459
1298336,0.0,8.762088e-01,1.088813,0.643104,5.549581e-01,-1.366025,0.57735,-0.993915,0.666885,0.523717,0.428459
1298337,0.0,1.393847e+00,-0.398533,-0.356896,-4.450419e-01,-1.366025,0.57735,-0.993915,0.666885,0.523717,0.428459
1298338,0.0,-4.226497e-01,-0.214413,-0.356896,-4.450419e-01,-1.366025,0.57735,-0.059243,-0.258178,-0.495814,-0.557575


In [61]:
Xa.isnull().any()

PUweekend     False
PUhoursin     False
PUhourcos     False
PUdaysin      False
PUdaycos      False
PUmonthsin    False
PUmonthcos    False
tojfk         False
tolgd         False
toql          False
tomar         False
dtype: bool

In [121]:
Xtrain,Xtest,ytrain,ytest= train_test_split(Xa,ya,test_size=0.2, random_state= 0)

In [122]:
# Which model, which hyperparameters to use? I do Cross-validation to check
modelhya= SGDRegressor(random_state=0)
hyperparameters= {'loss':['squared_loss','huber','epsilon_insensitive'], 'alpha': np.linspace(0,0.001, 3)} 
#,'squared_epsilon_insensitive'

In [123]:
grid= GridSearchCV(modelhya, hyperparameters, cv=5, scoring='neg_root_mean_squared_error',n_jobs=-1 )

In [124]:
grid.fit(Xtrain,ytrain)

GridSearchCV(cv=5, estimator=SGDRegressor(random_state=0), n_jobs=-1,
             param_grid={'alpha': array([0.    , 0.0005, 0.001 ]),
                         'loss': ['squared_loss', 'huber',
                                  'epsilon_insensitive']},
             scoring='neg_root_mean_squared_error')

In [125]:
comparison = pd.DataFrame(np.abs(grid.cv_results_['mean_test_score']),columns=['Loss'],index=['squared0','huber0','ep0','squared0.0005','huber0.0005','ep.0005','squared0.001','huber0.001','ep0.001'])
# this score is defined by scoring. Both hypothesis maps use different loss function(squared vs squared_epsilon) to come up with the best estimate. How to tell which one performs better? Set a common scoring criteria then (working the diff between y_true and y_pred(based on best estimate of each hypothesis map. I.g: SimpleLR vs SVR)) 


In [126]:
comparison #=> alpha =0, loss= squared is the best( see the results on real queens dataset). USE THESE HYPERPARAMETER FOR 3a ENTIRE DATASET

,Loss
squared0,0.550262
huber0,0.565723
ep0,0.563332
squared0.0005,0.550727
huber0.0005,0.562800
ep.0005,0.563099
squared0.001,0.551118
huber0.001,0.560948
ep0.001,0.562844


In [127]:
modela =SGDRegressor(alpha= 0, random_state=0)

In [128]:
modela.fit(Xtrain,ytrain)

SGDRegressor(alpha=0, random_state=0)

In [129]:
modela.score(Xtrain,ytrain)

0.35547338439861276

In [130]:
modela.score(Xtest,ytest)

0.3552761906349088

#### 3b Predicting fare_amount (parsimonious model)

In [78]:
yb= Xscaled['fare_amount']

In [79]:
Xscaled.columns

Index(['PUweekend', 'DOweekend', 'passenger_count', 'trip_distance',
       'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount',
       'tolls_amount', 'improvement_surcharge', 'total_amount',
       'congestion_surcharge', 'duration_sec', 'r2', 'r3', 'r4', 'r5', 'r6',
       'pay2', 'pay3', 'pay4', 'PUhoursin', 'PUhourcos', 'DOhoursin',
       'DOhourcos', 'PUdaysin', 'PUdaycos', 'DOdaysin', 'DOdaycos',
       'PUmonthsin', 'PUmonthcos', 'DOmonthsin', 'DOmonthcos', 'tojfk',
       'tolgd', 'toql', 'tomar', 'pd'],
      dtype='object')

In [80]:
Xb =Xscaled.drop(['fare_amount','payment_type',
                      'extra',
                     'mta_tax',            'tip_amount',
                'tolls_amount', 'improvement_surcharge',
                'total_amount',  'congestion_surcharge','trip_distance','duration_sec'
                ],axis=1)

In [81]:
Xb.columns

Index(['PUweekend', 'DOweekend', 'passenger_count', 'r2', 'r3', 'r4', 'r5',
       'r6', 'pay2', 'pay3', 'pay4', 'PUhoursin', 'PUhourcos', 'DOhoursin',
       'DOhourcos', 'PUdaysin', 'PUdaycos', 'DOdaysin', 'DOdaycos',
       'PUmonthsin', 'PUmonthcos', 'DOmonthsin', 'DOmonthcos', 'tojfk',
       'tolgd', 'toql', 'tomar', 'pd'],
      dtype='object')

In [82]:
Xb

,PUweekend,DOweekend,passenger_count,r2,r3,r4,r5,r6,pay2,pay3,...,DOdaycos,PUmonthsin,PUmonthcos,DOmonthsin,DOmonthcos,tojfk,tolgd,toql,tomar,pd
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.019377e-01,0.000000,0.42265,0.000000,0.42265,0.092201,0.000000,0.065138,0.134524,-0.700253
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-4.450419e-01,0.000000,0.42265,0.000000,0.42265,-0.993915,0.666885,0.523717,0.428459,0.493520
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-4.450419e-01,0.000000,0.42265,0.000000,0.42265,-0.993915,0.666885,0.523717,0.428459,0.733484
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-4.450419e-01,0.000000,0.42265,0.000000,0.42265,-0.993915,0.666885,0.523717,0.428459,1.498731
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,-4.450419e-01,0.000000,0.42265,0.000000,0.42265,0.006085,-0.333115,-0.476283,-0.571541,0.580578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1298335,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,-1.638616e-16,-1.366025,0.57735,-1.366025,0.57735,-0.993915,0.666885,0.523717,0.428459,0.259234
1298336,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,5.549581e-01,-1.366025,0.57735,-1.366025,0.57735,-0.993915,0.666885,0.523717,0.428459,0.298523
1298337,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-4.450419e-01,-1.366025,0.57735,-1.366025,0.57735,-0.993915,0.666885,0.523717,0.428459,0.905998
1298338,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-4.450419e-01,-1.366025,0.57735,-1.366025,0.57735,-0.059243,-0.258178,-0.495814,-0.557575,0.231988


In [89]:
Xtrain,Xtest, ytrain,ytest = train_test_split(Xb,yb,test_size=0.2, random_state= 0)

In [90]:
# Which model, which hyperparameters to use? I do the Cross-validation to check
modelhyb= SGDRegressor(random_state=0)
hyperparameters= {'loss':['squared_loss','huber'], 'alpha': [0,0.0001]} 


In [91]:
grid= GridSearchCV(modelhyb, hyperparameters, cv=5, scoring='neg_root_mean_squared_error',n_jobs=-1 )

In [92]:
grid.fit(Xtrain,ytrain)

GridSearchCV(cv=5, estimator=SGDRegressor(random_state=0), n_jobs=-1,
             param_grid={'alpha': [0, 0.0001],
                         'loss': ['squared_loss', 'huber']},
             scoring='neg_root_mean_squared_error')

In [93]:
comparison = pd.DataFrame(np.abs(grid.cv_results_['mean_test_score']),columns=['Loss'],index=['squared0','huber0','squared0.0001','huber0.0001']) 
# this score is defined by scoring. Both hypothesis maps use different loss function(squared vs squared_epsilon) to come up with the best estimate. How to tell which one performs better? Set a common scoring criteria then (working the diff between y_true and y_pred(based on best estimate of each hypothesis map. I.g: SimpleLR vs SVR)) 


In [94]:
comparison #=> alpha = 0, loss= squared_loss is the best. USE THESE HYPERPARAMETER FOR 3a ENTIRE DATASET

,Loss
squared0,0.240418
huber0,0.245564
squared0.0001,0.240493
huber0.0001,0.245689


In [103]:
# doing cross validation to find out how many features to go for
from sklearn.model_selection import cross_val_score
modelb = SGDRegressor(alpha=0,random_state=0)
scores = cross_val_score(modelb, Xtrain, ytrain, cv=5)


In [104]:
np.mean(scores)

0.8462813268190355

In [97]:
Xscaled.corr()


,PUweekend,DOweekend,passenger_count,trip_distance,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,...,DOdaycos,PUmonthsin,PUmonthcos,DOmonthsin,DOmonthcos,tojfk,tolgd,toql,tomar,pd
PUweekend,1.000000,0.987598,-0.001633,-0.025917,-0.012644,-0.043752,-0.110721,-0.000437,-0.008308,-0.033410,...,0.349814,-0.005170,-0.025021,-0.005217,-0.024966,0.010741,-0.011970,-0.012401,-0.012031,-0.022586
DOweekend,0.987598,1.000000,-0.001531,-0.025622,-0.012179,-0.043368,-0.110810,-0.000469,-0.008692,-0.033426,...,0.347213,-0.006306,-0.024878,-0.006252,-0.025094,0.010733,-0.011691,-0.012009,-0.011563,-0.022415
passenger_count,-0.001633,-0.001531,1.000000,0.033268,0.000510,0.033085,-0.029491,0.000840,0.004863,0.027221,...,-0.006879,0.011490,0.020921,0.011504,0.020899,-0.019069,0.014618,0.012694,0.011315,0.032748
trip_distance,-0.025917,-0.025622,0.033268,1.000000,-0.220745,0.972349,0.232201,-0.058064,0.454838,0.368564,...,-0.016043,0.061001,0.082637,0.060991,0.082669,-0.546426,0.476181,0.425487,0.378094,0.931456
payment_type,-0.012644,-0.012179,0.000510,-0.220745,1.000000,-0.232753,-0.116496,0.000480,-0.693382,-0.267997,...,-0.026534,-0.022681,-0.038807,-0.022700,-0.038755,-0.098346,0.131009,0.144966,0.155930,-0.224509
fare_amount,-0.043752,-0.043368,0.033085,0.972349,-0.232753,1.000000,0.230324,-0.112173,0.469269,0.381029,...,-0.028359,0.075045,0.087901,0.075030,0.087937,-0.498750,0.428641,0.371455,0.323465,0.911233
extra,-0.110721,-0.110810,-0.029491,0.232201,-0.116496,0.230324,1.000000,0.027046,0.200846,0.258635,...,-0.057395,0.046704,0.038652,0.046696,0.038667,-0.071810,0.059287,0.050278,0.042986,0.252014
mta_tax,-0.000437,-0.000469,0.000840,-0.058064,0.000480,-0.112173,0.027046,1.000000,-0.026383,-0.032828,...,0.001077,-0.004058,-0.003855,-0.004064,-0.003848,0.021339,-0.020478,-0.017701,-0.015786,-0.054934
tip_amount,-0.008308,-0.008692,0.004863,0.454838,-0.693382,0.469269,0.200846,-0.026383,1.000000,0.378435,...,0.010816,0.086154,0.081612,0.086139,0.081601,-0.089918,0.034848,0.005100,-0.018123,0.439971
tolls_amount,-0.033410,-0.033426,0.027221,0.368564,-0.267997,0.381029,0.258635,-0.032828,0.378435,1.000000,...,-0.014339,0.107124,0.084922,0.107111,0.084933,0.118612,-0.190096,-0.225016,-0.251362,0.346404


##### 6 strongest features: pd + 4 distance features + r2

In [105]:
modelb6 = SGDRegressor(alpha=0,random_state=0)
scoresb6 = cross_val_score(modelb6, Xtrain[['pd','tojfk',
       'tolgd', 'toql', 'tomar','r2' ]], ytrain, cv=5)

In [106]:
np.mean(scoresb6)

0.8346969780336648

##### Could the model be more parsimonious? 1 feature only. PU DO distance

In [107]:
modelb1= SGDRegressor(alpha=0,random_state=0)
scoresb1 = cross_val_score(modelb1, pd.DataFrame(Xtrain['pd'],index=Xtrain.index,columns=['pd']), ytrain, cv=5)

In [108]:
np.mean(scoresb1)

0.8291611052916144

####### => conclusion: 1 feature should be best both in terms of the performance of the model and the number of parameters

In [109]:
finalmodelb= SGDRegressor(alpha=0,random_state=0)
finalmodelb.fit(Xtrain[['pd']],ytrain)

SGDRegressor(alpha=0, random_state=0)

In [110]:
finalmodelb.score(Xtest[['pd']],ytest)

0.8349994900792068

#### 3c Predict payment type

In [111]:
Xscaled.columns

Index(['PUweekend', 'DOweekend', 'passenger_count', 'trip_distance',
       'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount',
       'tolls_amount', 'improvement_surcharge', 'total_amount',
       'congestion_surcharge', 'duration_sec', 'r2', 'r3', 'r4', 'r5', 'r6',
       'pay2', 'pay3', 'pay4', 'PUhoursin', 'PUhourcos', 'DOhoursin',
       'DOhourcos', 'PUdaysin', 'PUdaycos', 'DOdaysin', 'DOdaycos',
       'PUmonthsin', 'PUmonthcos', 'DOmonthsin', 'DOmonthcos', 'tojfk',
       'tolgd', 'toql', 'tomar', 'pd'],
      dtype='object')

In [112]:
yc= Xscaled['payment_type']

In [113]:
Xc=Xscaled.drop(['payment_type', 'pay2', 'pay3', 'pay4'],axis=1)

In [114]:
Xtrain,Xtest, ytrain,ytest = train_test_split(Xc,yc,test_size=0.2, random_state=0)

In [115]:
Xtrain.columns

Index(['PUweekend', 'DOweekend', 'passenger_count', 'trip_distance',
       'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount', 'congestion_surcharge',
       'duration_sec', 'r2', 'r3', 'r4', 'r5', 'r6', 'PUhoursin', 'PUhourcos',
       'DOhoursin', 'DOhourcos', 'PUdaysin', 'PUdaycos', 'DOdaysin',
       'DOdaycos', 'PUmonthsin', 'PUmonthcos', 'DOmonthsin', 'DOmonthcos',
       'tojfk', 'tolgd', 'toql', 'tomar', 'pd'],
      dtype='object')

In [116]:
Xtrain.isnull().any()

PUweekend                False
DOweekend                False
passenger_count          False
trip_distance            False
fare_amount              False
extra                    False
mta_tax                  False
tip_amount               False
tolls_amount             False
improvement_surcharge    False
total_amount             False
congestion_surcharge     False
duration_sec             False
r2                       False
r3                       False
r4                       False
r5                       False
r6                       False
PUhoursin                False
PUhourcos                False
DOhoursin                False
DOhourcos                False
PUdaysin                 False
PUdaycos                 False
DOdaysin                 False
DOdaycos                 False
PUmonthsin               False
PUmonthcos               False
DOmonthsin               False
DOmonthcos               False
tojfk                    False
tolgd                    False
toql    

In [117]:
modelc= SGDClassifier(loss='log',random_state=0)



In [118]:
modelc.fit(Xtrain,ytrain)

SGDClassifier(loss='log', random_state=0)

In [119]:
modelc.score(Xtrain,ytrain)


0.95111851521332

In [120]:
modelc.score(Xtest,ytest)

0.9507790740043107